In [ ]:
import csv
import os
import torch

import numpy as np
import pandas as pd

from multiprocessing import cpu_count
from multiprocessing.pool import Pool
from pathlib import Path

from classes.dataset import CustomDataset

In [ ]:
dataset_path = Path('dataset')
csv_file = dataset_path/'metadata.csv'

# Class PreProcessor

In [ ]:
file = open(csv_file)

In [ ]:
lines = [line.rstrip() for line in file.readlines()]
print(lines)

In [ ]:
sample_dim = [int(element) for element in lines[0].split(",")]
print(sample_dim)

metadata = []
for l in lines[1:]:
    line = l.split(",")
    name = line[0]
    for i in range(1,len(line)):
        line[i] = int(line[i])
    metadata.append(line)

print(metadata)

In [ ]:
scaled_coordinates = dataset_path/'scaled_coordinates'
os.makedirs(scaled_coordinates, exist_ok=True)

def preprocess(meta_line):
    name = meta_line[0]
    name = os.path.splitext(name)[0] # remove extension
    
    coordinates = meta_line[1:]
    for i in range(0, len(coordinates), 2):
        coordinates[i] /= sample_dim[0]
        coordinates[i+1] /= sample_dim[1]
    
    array_coord = np.array([coordinates])
    np.save(scaled_coordinates/f'{name}.npy', array_coord, allow_pickle=False)
    return name

In [ ]:
#It seems pooling does not work on notebooks
pool = Pool(processes=cpu_count())
mapper = pool.imap_unordered(preprocess, metadata)

In [ ]:
for l in metadata:
    preprocess(l)

# Pandas exploration (unused)

In [ ]:
sample_dim = pd.read_csv(csv_file, header=None, nrows=1).values.tolist()[0]
img_labels = pd.read_csv(csv_file, header=None, skiprows=1)

In [ ]:
print(sample_dim)
print(img_labels)

In [ ]:
customDS = CustomDataset(dataset_path, csv_file)

In [ ]:
print(customDS.__getitem__(0)['label'])

In [ ]:
sample_dim = pd.read_csv(csv_file, header=None, nrows=1).values.tolist()[0]
img_labels = pd.read_csv(csv_file, header=None, skiprows=1)
img_labels.rename(columns={0:'Name'}, inplace=True)

print(sample_dim)
img_labels

In [ ]:
for index in img_labels.index:
    #print(f'index : {index}')
    #print(img_labels.loc[index].tolist())
    row = img_labels.loc[index].tolist()

    name = row[0]
    coordinates = row[1:]

#img_labels.loc[:, 1::2].apply(axis=1, inplace=True, func=lambda x: x/sample_dim[0],)
#img_labels.loc[:, 2::2].apply(axis=1, inplace=True, func=lambda x: x/sample_dim[1])

for i in range(1, len(img_labels.columns)):
    img_labels[i] = img_labels[i]/sample_dim[(i+1)%2]

In [ ]:
img_labels[img_labels.columns[:]]

In [ ]:
coordinates = img_labels.iloc[0, 1:].tolist()
print(coordinates)

In [ ]:
for i in range(len(img_labels.index)):
    coordinates = img_labels[1:].tolist()